In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask,request,render_template
import pickle
import tensorflow as tf
from tensorflow import keras
import os

import numpy as np
import cv2
from PIL import Image

from flask_ngrok import run_with_ngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab.patches import cv2_imshow
from flask import Flask
from werkzeug.utils import secure_filename
app=Flask(__name__,template_folder='/content/drive/MyDrive/Colab Notebooks/templates')
run_with_ngrok(app)   
model_path = "/content/drive/My Drive/Colab Notebooks/identify_sign.h5"
model = keras.models.load_model(model_path)
@app.route('/')
def r():
  return render_template("index.html")

@app.route('/success',methods = ['POST'])
def home():
    file = request.files['file'].read() ## byte file
    npimg=np.fromstring(file,np.uint8)
    img=cv2.imdecode(npimg,cv2.IMREAD_COLOR)
    def orb_sim(img1, img2):

  # SURF is no longer available in cv2 so using ORB
      orb = cv2.ORB_create()

  # detect keypoints and descriptors
      kp_a, desc_a = orb.detectAndCompute(img1, None)
      kp_b, desc_b = orb.detectAndCompute(img2, None)

  # define the bruteforce matcher object
      bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
  #perform matches. 
      matches = bf.match(desc_a, desc_b)
  #Look for similar regions with distance < 50. Goes from 0 to 100 so pick a number between.
      similar_regions = [i for i in matches if i.distance < 60]  
  # if len(matches) == 0:
  #   return 0
      return len(similar_regions) / len(matches)
    #cv2_imshow(img)
    #image=cv2.imread(image1)
    #image = cv2.imread("/content/drive/My Drive/Signature_classify/test/User1/00102001.png")
    image_fromarray = Image.fromarray(img, 'RGB')
    resize_image = image_fromarray.resize((128, 128))
    expand_input = np.expand_dims(resize_image,axis=0)
    input_data = np.array(expand_input)
    input_data = input_data/255
    pred = model.predict(input_data)
    result = pred.argmax()
    input_user = result + 1
    #print('The Signature Belongs to: User' + str(input_user))
    #return render_template("index.html",content='The signature belongs to '+' '+str(input_user))
    original_signatures = []
    input_image = img
    for i in range(5):

     original_signatures.append("/content/drive/My Drive/Signature_classify/train/User"+ str(input_user)+"/00"+str(input_user)+"0"+str(i)+"00"+str(input_user)+".png")
    max_identical = 0
    orb_similarity = 0
    for i in range(len(original_signatures)):
     original_signature = cv2.imread(original_signatures[i])
     orb_similarity = orb_sim(original_signature, input_image)
     if max_identical < orb_similarity:
       max_identical = orb_similarity


    print("Similarity using ORB is: ", max_identical)
    if max_identical > 0.7:
      return render_template("index.html",content='The signature is genuine')
    else:
      return render_template("index.html",content='The signature is forged')

#print(original_signatures)
    
	  
	  













    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d712-34-125-0-185.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/Apr/2022 07:18:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2022 07:18:54] "GET /node_modules/bootstrap/dist/css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 07:18:54] "GET /node_modules/bootstrap-social/bootstrap-social.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 07:18:54] "GET /node_modules/font-awesome/css/font-awesome.min.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 07:18:55] "GET /style.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 07:18:55] "GET /SigDetectCo-logos.jpeg HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 07:18:56] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  from ipykernel import kernelapp as app
127.0.0.1 - - [28/Apr/2022 07:19:06] "POST /success HTTP/1.1" 200 -


Similarity using ORB is:  1.0


...

127.0.0.1 - - [28/Apr/2022 08:56:18] "GET /node_modules/bootstrap/dist/css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 08:56:18] "GET /node_modules/bootstrap-social/bootstrap-social.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 08:56:18] "GET /node_modules/font-awesome/css/font-awesome.min.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 08:56:19] "GET /SigDetectCo-logos.jpeg HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 09:18:42] "POST /success HTTP/1.1" 200 -


Similarity using ORB is:  1.0


127.0.0.1 - - [28/Apr/2022 09:18:43] "GET /node_modules/bootstrap/dist/css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 09:18:43] "GET /style.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 09:18:43] "GET /node_modules/bootstrap-social/bootstrap-social.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 09:18:43] "GET /node_modules/font-awesome/css/font-awesome.min.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 09:18:43] "GET /SigDetectCo-logos.jpeg HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 09:19:01] "POST /success HTTP/1.1" 200 -


Similarity using ORB is:  0.3333333333333333


127.0.0.1 - - [28/Apr/2022 09:19:02] "GET /node_modules/bootstrap/dist/css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 09:19:02] "GET /node_modules/font-awesome/css/font-awesome.min.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 09:19:02] "GET /node_modules/bootstrap-social/bootstrap-social.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 09:19:02] "GET /style.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Apr/2022 09:19:02] "GET /SigDetectCo-logos.jpeg HTTP/1.1" 404 -


In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)   
  
@app.route("/home")
def home():
    return "<h1>GFG is great platform to learn</h1>"
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c5f2-35-229-87-76.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 7.7 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=8c763340c6599bc3a3a3de593395f0798a4d0f0b87f7cb787d28da74a5442788
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!ngrok authtoken 27pY82oXWGE73Pjf8D1iKLsM6H0_7qhLpCqiTyJYqb21vW2z9

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!bash

bash: cannot set terminal process group (61): Inappropriate ioctl for device
bash: no job control in this shell
/content# 

/content# 
/content# exit
